This notebook extracts information about time and location of spikes from the hdf5 file with the BrainGrid simulation data and converts it into a dataframe with the following columns: time_step (time step when the spike happened), xloc, yloc (coordinates of the spike). It then extracts spacial and temporal pre-burst and non-burst windows, labels them, and saves the result in a csv.

Author: Mariia Lundvall (lundvm@uw.edu) 

In [1]:
import numpy as np
import pandas as pd
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import h5py
from tqdm import tqdm_notebook as tqdm
import gc
import numpy.ma as ma
import boto3
import io
import gzip

In [2]:
def read_csv(filename, bucket='braingrid-data'):
    """
    Loads a csv file from s3. 
    
    Args:
        filename(str): the path (key) to the csv file.
        bucket(str): the name of the s3 bucket to load the data from. Default is 'braingrid-data'. 
        
    Returns:
        df(pandas df): loaded csv as a pandas dataframe.
    """
    
    s3 = boto3.client('s3') 
    obj = s3.get_object(Bucket= bucket, Key= filename) 
    df = pd.read_csv(obj['Body'])
    
    return df

In [3]:
def pandas_to_s3(df, bucket='braingrid-data', key):
    """
    Saves pandas dataframe to s3 as a gz file.
    
    Args:
        df(pandas dataframe): dataframe to save to s3
        bucket(str): the name of the bucket ('braingrid-data' by default).
        key(str): the file key (the path where to save the file + the file name)
    
    Returns:
        none
    """
    
    # create s3 client
    client = boto3.client('s3')
    
    # write DF to string stream
    print('creating csv buffer')
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # reset stream position
    csv_buffer.seek(0)
    # create binary stream
    gz_buffer = io.BytesIO()

    # compress string stream using gzip
    print('compressing')
    with gzip.GzipFile(mode='w', fileobj=gz_buffer) as gz_file:
        gz_file.write(bytes(csv_buffer.getvalue(), 'utf-8'))
        
    # write stream to S3
    print('writing to ss')
    obj = client.put_object(Bucket=bucket, Key=key, Body=gz_buffer.getvalue())

In [4]:
def create_spike_matrix(f):
    """
    Takes in an HDF5 file produced by the BrainGrid simuation, extracts information about time 
    and location of spikes and converts it into a dataframe with the following columns: time_step 
    (time step when the spike happened), xloc, yloc (coordinates of the spike).
    
    Args: 
        f(HDF5): loaded HDF5 file to read data from.
    
    Returns:
        spikes_loc(pandas dataframe): dataframe that contains spiking data. The dataframe has the 
            following columns: time_step (time step when the spike happened), xloc, yloc 
            (coordinates of the spike).
    """
    
    # get the spikes time steps and coordinates data from the hdf5 file and convert the data 
    # into numpy arrays
    spikes = np.array(f['/spikesProbedNeurons'], dtype='uint32')
    xloc = np.array(f['/xloc'], dtype='uint8')
    yloc = np.array(f['/yloc'], dtype='uint8')
    # m is the max number of spikes per neuron, n is the number of neurons in the simulation
    m, n = spikes.shape
    # transform the spikes matrix:
    # 1. Traspose so that each row is a sequence of spikes of one neuron (instead of a column)
    # 2. Flattem the matrix 
    # 3. Reshape into a 2d array from (m*n, ) to (m*n, 1). This is needed for further processing. 
    spikes = np.transpose(spikes).flatten().reshape(m*n, 1)
    # create a mask to remove non-spikes (where time step=0)
    mask = ma.masked_equal(spikes, 0).reshape(m*n, )
    # Transform the coordinate vectors:
    # 1. Make the vectors match the time step sequence. Repeat the values so that first m values 
    # in the x and y vectors are the coordinates of the first neuron
    # 2. Remove the coordinates corresponding to non-spikes
    # 3. Concatenate x and y, the result is an array xy of shape (m*n, 2)
    xloc = np.compress(mask, np.repeat(xloc, m).reshape(m*n, 1), axis=0)
    yloc = np.compress(mask, np.repeat(yloc, m).reshape(m*n, 1), axis=0)
    xy = np.concatenate((xloc, yloc), axis=1)
    # delete xloc and yloc to free memory
    del xloc
    del yloc
    gc.collect()
    # Remove non-spikes from the time step array, concatenate it to xy, 
    # and convert the result into a dataframe
    spikes_loc = pd.DataFrame(np.concatenate((np.compress(mask, spikes, axis=0), xy), axis=1))
    # delete spikes to free the memory
    del spikes
    gc.collect()
    spikes_loc.rename(columns={0:'time_step', 1:'xloc', 2:'yloc'}, inplace=True)
    spikes_loc.sort_values(by='time_step', inplace=True)
    spikes_loc.reset_index(drop=True, inplace=True)
    
    return spikes_loc

In [5]:
def get_training_data(windows, gap, spikes, x, y, mask, n):
    """
    Takes in the spiking data, extracts spacial and temporal pre-bursts and non-bursts 
    windows, adds a label (0 for non-burst, 1 for pre-burst) to each window, and saves 
    the windows as a csv (one csv per window size, separate csvs with temporal and spacial data).

    Args:
        windows(list of ints): the list containing window sizes
        gap(int): the size of the gap between the beginning of a burst and a non-burst window.
        spikes(numpy 1d array): array contaning spikes time steps
        x(numpy 1d array): array contaning x coordinates of the spikes
        y(numpy 1d array): array containing y coordinates of the spikes
        mask(boolean array): array of the same length as spikes, containing True for time steps 
            that indicate the starts of bursts
        n(int): number of identified bursts in the simulation

    Returns:
        none
        saves csv files with temporal and spacial training data
    """
    m = spikes.shape[0]
    for w in tqdm(windows):
        print('Extracting windows of size ' + str(w))
        # initialize pre-burst and non-burst arrays
        temp_preburst = np.ones((n, w+1))
        temp_nonburst = np.zeros((n, w+1))
        space_preburst = np.ones((n, w*2+1))
        space_nonburst = np.zeros((n, w*2+1))
        index = 0
        for i in range(1, m):
            if not mask[i-1] and mask[i]:
                # attempt to remove bad bursts. If the gap width is smaller than the
                # gap size, we must be catching the beginning of a burst, so don't extract those windows
                if spikes[i] - spikes[i-gap] >= gap:
                    # get the temporal windows
                    temp_preburst[index, 0:w] = spikes[i-w:i] - spikes[i-w]
                    temp_nonburst[index, 0:w] = spikes[i -
                                                       w-gap:i-gap] - spikes[i-w-gap]
                    # get the spacial windows
                    space_preburst[index, 0:w*2] = np.concatenate(
                        (x[i-w:i].reshape(w, 1), y[i-w:i].reshape(w, 1)), axis=1).flatten()
                    space_nonburst[index, 0:w*2] = np.concatenate((x[i-w-gap:i-gap].reshape(
                        w, 1), y[i-w-gap:i-gap].reshape(w, 1)), axis=1).flatten()
                    index += 1
        if index < n:
            temp_preburst = np.delete(temp_preburst, np.s_[index:n], 0)
            temp_nonburst = np.delete(temp_nonburst, np.s_[index:n], 0)
            space_preburst = np.delete(space_preburst, np.s_[index:n], 0)
            space_nonburst = np.delete(space_nonburst, np.s_[index+1:n], 0)

        # combine and shuffle data
        temp_traindata = np.vstack((temp_preburst, temp_nonburst)).astype(int)
        np.random.shuffle(temp_traindata)
        space_traindata = np.vstack(
            (space_preburst, space_nonburst)).astype(int)
        np.random.shuffle(space_traindata)
        # save the result as a csv
        np.savetxt('train_data/temp_' + str(w) + '.csv',
                   temp_traindata, delimiter=',')
        np.savetxt('train_data/space_' + str(w) + '.csv',
                   space_traindata, delimiter=',')
        print('Total bursts: ' + str(index))

In [6]:
#load an hdf5 file with the simulation data
f = h5py.File('tR_1.0--fE_0.90.h5', 'r')
print(list(f.keys()), '\n')
print(list(f.values()))

['Tsim', 'burstinessHist', 'neuronThresh', 'neuronTypes', 'probedNeurons', 'radiiHistory', 'ratesHistory', 'simulationEndTime', 'spikesHistory', 'spikesProbedNeurons', 'starterNeurons', 'xloc', 'yloc'] 

[<HDF5 dataset "Tsim": shape (1,), type "<f4">, <HDF5 dataset "burstinessHist": shape (60000,), type "<i4">, <HDF5 dataset "neuronThresh": shape (10000,), type "<f4">, <HDF5 dataset "neuronTypes": shape (10000,), type "<i4">, <HDF5 dataset "probedNeurons": shape (10000,), type "<i4">, <HDF5 dataset "radiiHistory": shape (601, 10000), type "<f4">, <HDF5 dataset "ratesHistory": shape (601, 10000), type "<f4">, <HDF5 dataset "simulationEndTime": shape (1,), type "<f4">, <HDF5 dataset "spikesHistory": shape (6000000,), type "<i4">, <HDF5 dataset "spikesProbedNeurons": shape (375898, 10000), type "<u8">, <HDF5 dataset "starterNeurons": shape (1000,), type "<i4">, <HDF5 dataset "xloc": shape (10000,), type "<i4">, <HDF5 dataset "yloc": shape (10000,), type "<i4">]


In [7]:
%%time
# create the spiking data matrix
spikes_loc = create_spike_matrix(f)

CPU times: user 7min 39s, sys: 3min 25s, total: 11min 4s
Wall time: 10min 24s


In [ ]:
%%time
# save the result to s3 (optional)
pandas_to_s3(spikes_loc, 'spikes.gz')

In [8]:
# read in burst data
bursts = read_csv('allBurst.csv')['StartT'].values
# initialize variables
windows = [5, 10, 50, 100, 500]
gap = 2000
mask = np.in1d(spikes_loc.values[:, 0], bursts)
n = bursts.shape[0]
# get the training data
get_training_data(
    windows, gap, spikes_loc.values[:, 0], spikes_loc.values[:, 1], spikes_loc.values[:, 2], mask, n)

Extracting windows of size 5
Total bursts: 9710
Extracting windows of size 10
Total bursts: 9710
Extracting windows of size 50
Total bursts: 9710
Extracting windows of size 100
Total bursts: 9710
Extracting windows of size 500
Total bursts: 9710

